In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
from scipy.optimize import basinhopping, minimize
from tqdm.auto import tqdm

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

<Figure size 640x480 with 0 Axes>

In [8]:
P0 = lambda n, theta: np.cos((2*n+1)*theta)**2
P1 = lambda n, theta: np.sin((2*n+1)*theta)**2

def estimate_signal(depths, n_samples, theta):
        signals = np.zeros(len(depths), dtype = np.complex128)
        measurements = np.zeros(len(depths))
        for i,n in enumerate(depths):
            # Get the exact measuremnt probabilities
            p0 = P0(n, theta)
            p1 = P1(n, theta)

            p0x = P0x(n,theta)
            p1x = P1x(n,theta)
            
            p0_estimate = np.random.binomial(n_samples[i], p0)/n_samples[i]
            p1_estimate = 1 - p0_estimate

            p0x_estimate = np.random.binomial(n_samples[i], p0x)/n_samples[i]
            p1x_estimate = 1.0 - p0x_estimate
            measurements[i] = p0_estimate
            
            # Estimate theta
            theta_estimated = np.arctan2(p0x_estimate - p1x_estimate, p0_estimate - p1_estimate)

            # Compute f(n) - Eq. 3
            fi_estimate = np.exp(1.0j*theta_estimated)
            signals[i] = fi_estimate
         
        return signals, measurements

from scipy.stats import binom

def apply_correction(ula_signal, measurements, theta_est, theta):
    theta_est = np.abs(theta_est)
    p_exact = np.cos((2*ula_signal.depths+1)*(theta))**2
    p_neg = np.cos((2*ula_signal.depths+1)*(-theta))**2
    p_o2 = np.cos((2 * ula_signal.depths + 1) * (theta_est/2.0)) ** 2
    p_o4 = np.cos((2 * ula_signal.depths + 1) * (theta_est/4.0)) ** 2
    p_same = np.cos((2*ula_signal.depths+1)*(theta_est))**2
    p_s2 = np.cos((2 * ula_signal.depths + 1) * (np.pi/2-theta_est)) ** 2
    p_s4 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est)) ** 2
    p_s2_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 2 - theta_est/2)) ** 2
    p_s4_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est/2)) ** 2

    l_exact = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_exact[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_neg = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_neg[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk]*measurements[kk], ula_signal.n_samples[kk], p_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_o4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_o4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_same = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    
    which_correction = np.argmax([l_same, l_s2, l_s4, l_o2, l_o4, l_s2_o2, l_s4_o2])
    if which_correction == 1:
        theta_est = np.pi/2.0 - theta_est
    elif which_correction == 2:
        theta_est = np.pi/4.0 - theta_est
    elif which_correction == 3:
        theta_est = theta_est/2
    elif which_correction == 4:
        theta_est = theta_est/4
    elif which_correction == 5:
        theta_est = np.pi / 2.0 - 0.5 * theta_est
    elif which_correction == 6:
        theta_est = np.pi / 4.0 - 0.5 * theta_est
    # elif which_correction == 7:
    #     theta_est = -theta_est

    # print(f'FINAL ANGLE FOUND: {theta_est, theta}')

    return np.abs(theta_est)

In [9]:
import numpy as np
import itertools
from typing import List

def generate_all_sign_variations(signs_array: np.ndarray) -> np.ndarray:
    """
    Generate all possible sign variations by changing signs at all possible pairs of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Get all possible unique pairs of positions
    n = len(signs_array)
    position_pairs = list(itertools.combinations(range(n), 2))
    
    # Will store all variations
    all_variations = []
    
    # Iterate through all possible position pairs
    for pos1, pos2 in position_pairs:
        # Generate variations for this pair of positions
        pair_variations = generate_pair_variations(signs_array, pos1, pos2)
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_adjacent_sign_variations(signs_array: np.ndarray, size: int) -> np.ndarray:
    """
    Generate sign variations by sliding a two-position window across the array.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Total length of the array
    n = len(signs_array)
    
    # Will store all variations
    all_variations = []
    
    # Iterate through adjacent position pairs 
    # (0,1), (1,2), (2,3), ... until the second-to-last pair
    for pos1 in range(1, n - size + 1):
        pos = [pos1 + i for i in range(size)]
        
        # Generate variations for this pair of adjacent positions
        pair_variations = generate_pair_variations(signs_array, pos)
 
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_pair_variations(signs_array: np.ndarray, pos: List[int]) -> List[np.ndarray]:
    """
    Generate sign variations for a specific pair of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs
    pos1 : int
        First position to modify
    pos2 : int
        Second position to modify
    
    Returns:
    --------
    List[numpy.ndarray]
        List of sign variation arrays
    """
    # Validate input positions
    # if pos1 < 0 or pos2 < 0 or pos1 >= len(signs_array) or pos2 >= len(signs_array):
    #     raise ValueError("Positions must be within the array bounds")
    
    # Generate all possible sign combinations for the two positions
    sign_combinations = list(itertools.product([-1, 1], repeat=len(pos)))
    
    # Create variations
    variations = []
    for combo in sign_combinations:
        # Create a copy of the original array
        variation = signs_array.copy()
        
        # Modify the two specified positions
        for i in range(len(combo)):
            variation[pos[i]] *= combo[i]
        
        variations.append(tuple(variation))
    
    return variations


# Example array
signs = np.array([1]*4)

# Generate variations for all position pairs
all_variations = generate_adjacent_sign_variations(signs, 3)

# print("Original array:", signs)
# print(f"\nTotal variations: {len(all_variations)}")
# print(f'4*len(signs)^2: {4*len(signs)**2}')
# print(f'2^len(signs): {2**len(signs)}')

print("\nFirst few variations:")
for var in all_variations:  # Print first 10 variations
    print(var)


First few variations:
(1, -1, -1, -1)
(1, -1, -1, 1)
(1, -1, 1, -1)
(1, -1, 1, 1)
(1, 1, -1, -1)
(1, 1, -1, 1)
(1, 1, 1, -1)
(1, 1, 1, 1)


## Define the objective function to minimize

In [10]:
def objective_function_eig(lp, cos_signal, abs_sin, ula_signal, esprit):   
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    _, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    obj = eigs[1] - eigs[0]

    return obj

def objective_function(lp, cos_signal, abs_sin, ula_signal, esprit):
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)

    theta_est = apply_correction(ula_signal, ula_signal.measurements, theta_est, theta_est)

    # print(f'2*theta_est: {2*theta_est}')
    p_same = np.cos((2 * ula_signal.depths + 1) * (theta_est)) ** 2

    obj = -np.sum(
        np.log(
            [1e-75 + binom.pmf(ula_signal.n_samples[kk] * ula_signal.measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk
             in
             range(len(ula_signal.n_samples))]))


    # eigs = np.abs(esprit.eigs)[:2]
    # obj = eigs[1] - eigs[0]

    return obj

## What is the distribution of the signs? 

In [11]:
narray = [2,2,2,2,2,3]

def get_heavy_signs(narray, steps):

    thetas = np.linspace(np.arcsin(0.09), np.arcsin(0.91), steps)
    avals = np.sin(thetas)

    ula_signal = TwoqULASignal(M=narray, C=3)

    num_mc = 1000

    sign_distributions = {}

    for a,theta in zip(avals,thetas):
        # theta = np.arcsin(a)
        distr = {}
        for i in range(num_mc):
            signal, _ = estimate_signal(ula_signal.depths, ula_signal.n_samples, theta)
                # print(measurements)
            signs = tuple(np.sign(np.imag(signal)))
            # print(signs)

            distr[signs] = distr.get(signs, 0.0) + 1/num_mc
        sign_distributions[str(a)] = distr

    def get_signs(sign_distribution):
        # Sort the dictionary by values in descending order
        sorted_data = dict(sorted(sign_distribution.items(), key=lambda item: item[1], reverse=True))

        # Create a new dictionary with cumulative sum close to 0.68
        cumulative_dict = {}
        cumulative_sum = 0.0

        for key, value in sorted_data.items():
            cumulative_dict[key] = value
            if cumulative_sum + value > 0.9:
                break
            cumulative_sum += value

        # Output the result
        return cumulative_dict
    
    ret_dict = {}
    for a in avals:
        distr = get_signs(sign_distributions[str(a)])
        ret_dict[str(a)] = distr
 
    # Normalize the values
    normalized_data = {}
    for key, sub_dict in ret_dict.items():
        total_sum = sum(sub_dict.values())
        normalized_data[key] = {k: v / total_sum for k, v in sub_dict.items()}

    return normalized_data

normalized_data = get_heavy_signs(narray, 10)

# normalized_data

In [12]:
import ast
# Taking a random sample based on the normalized probabilities
def sample_from_normalized(data, sample_size=1):
    keys = list(map(str, list(data.keys())))
    probabilities = list(data.values())
    sampled_keys = np.random.choice(a=keys, size=sample_size, p=probabilities)
    return [ast.literal_eval(t) for t in sampled_keys]
avals = list(normalized_data.keys())
# Get one random sample
sampled = sample_from_normalized(normalized_data[avals[0]], sample_size=3)
# print("Sampled keys:",sampled)

## Use the distribution of signs so we don't have to try all possible signs

In [17]:
np.random.seed(42)

a = 0.1
print(a)
theta = np.arcsin(a)

narray = [2,2,2,2,2,2,2,2]
heavy_signs = get_heavy_signs(narray, len(narray)**2)

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

def sample_signs(heavy_signs, sample_size=3):
    """
    Sample a set of sign variations from a learned sign distribution.

    Args:
        heavy_signs (dict): A dictionary where the keys are strings representing float values,
                           and the values are dictionaries with keys as sign vectors and values
                           as their corresponding probabilities.
        sample_size (int, optional): The number of sign variations to sample. Defaults to 3.

    Returns:
        dict: A dictionary where the keys are the same as the keys in `heavy_signs`,
              and the values are lists of sampled sign variations.
    """
    avals = list(heavy_signs.keys())
    signs_to_try = {}
    for a in avals:
        signs_to_try[a] = list(set(sample_from_normalized(heavy_signs[a], sample_size=sample_size)))

    return signs_to_try

def avals_to_usef(a0, avals, L=3):
    """
    Find the `L` values in `avals` that are closest to the given `a0` value.

    Args:
        a0 (float): The reference value to find the closest `L` values for.
        avals (list): A list of float values representing the keys in `heavy_signs`.
        L (int, optional): The number of closest values to return. Defaults to 3.

    Returns:
        list: A list of string representations of the `L` closest values to `a0` in `avals`.
    """
    avals = list(map(float, avals))

    left = 0
    right = len(avals)
    
    while left < right:
        mid = (left + right) // 2
        
        if avals[mid] <= a0:
            left = mid + 1
        else:
            right = mid

    if left >= 4:
        avals_to_use = list(map(str, avals[left-L: left+L]))
    if left >= 3:
        avals_to_use = list(map(str, avals[left-3: left+L]))
    elif left>=2:
        avals_to_use = list(map(str, avals[left-2: left+L]))
    elif left>=1:
        avals_to_use = list(map(str, avals[left-1: left+L]))
    else:
        avals_to_use = list(map(str, avals[left: left+L]))
    
    return avals_to_use

def all_signs_to_try(avals_to_use, signs_to_try, adjacency=2):
    """
    Generate all possible sign variations within a Hamming distance of 2 from the given `avals_to_use`.

    Args:
        avals_to_use (list): A list of string representations of float values.
        signs_to_try (dict): A dictionary where the keys are string representations of float values,
                            and the values are lists of sign variations.
        adjacency (int, optional): The maximum Hamming distance to consider for the sign variations.
                                  Defaults to 2.

    Returns:
        list: A list of all possible sign variations within the specified Hamming distance.
    """
    all_signs = []
    for a in avals_to_use:
        for x in signs_to_try[a]:
            hamming_distance_two_signs = generate_adjacent_sign_variations(np.array(x), adjacency)
            all_signs.extend(hamming_distance_two_signs)
    all_signs = list(set(all_signs))
    
    return all_signs

def minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp):
    """
    Find the sign variation that minimizes the objective function.

    Args:
        all_signs (list): A list of all possible sign variations to consider.
        cos_signal (numpy.ndarray): The real part of the estimated signal.
        abs_sin (numpy.ndarray): The absolute value of the imaginary part of the estimated signal.
        ula_signal (ULASignal): An instance of the ULASignal class, containing information about the signal.
        esprit (ESPRIT): An instance of the ESPRIT class, used for estimating the signal parameters.
        disp (bool): If True, prints the current objective and the current best sign variation.

    Returns:
        numpy.ndarray: The sign variation that minimizes the objective function.
    """
    # obj = 1.0
    x_star = np.array(all_signs[0])
    obj = objective_function(x_star, cos_signal, abs_sin, ula_signal, esprit)
    for x in all_signs:
        curr_obj = objective_function(np.array(x), cos_signal, abs_sin, ula_signal, esprit)
        if curr_obj < obj:
            if disp:
                print(f'current objective: {curr_obj}')
                print(f'current best signs: {x}')
            obj = curr_obj
            x_star = np.array(x)

    return x_star

def csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=False, correction=False, optimize=False, disp=False):
    """
    Perform CSAE (Compressive Sensing Angle Estimation) with local minimization.

    Args:
        theta (float): The true angle of the signal.
        ula_signal (ULASignal): An instance of the ULASignal class, containing information about the signal.
        esprit (ESPRIT): An instance of the ESPRIT class, used for estimating the signal parameters.
        heavy_signs (dict): A dictionary where the keys are strings representing float values,
                           and the values are dictionaries with keys as sign vectors and values
                           as their corresponding probabilities.
        sample (bool, optional): If True, samples sign variations from the learned sign distribution.
                                If False, uses the learned sign distribution directly. Defaults to False.
        correction (bool, optional): If True, applies a correction to the estimated angle. Defaults to False.
        optimize (bool, optional): If True, uses a sliding window approach to find the best sign variation.
                                  If False, uses the learned sign distribution directly. Defaults to False.
        disp (bool, optional): If True, prints additional information during the process. Defaults to False.

    Returns:
        dict: A dictionary containing the estimated angles, errors, number of queries, maximum depth,
              and other relevant information.
    """

    depths = ula_signal.depths
    n_samples = ula_signal.n_samples
    csignal, measurements = estimate_signal(depths, n_samples, theta)
    ula_signal.measurements = measurements

    cos_signal = np.real(csignal)

    correct_signs = np.sign(np.imag(csignal))
    if disp:
        print(f'correct signs: {correct_signs}')
    abs_sin = np.abs(np.imag(csignal))

    if sample:
        # step 1: sample signs from learned sign distribution
        signs_to_try = sample_signs(heavy_signs=heavy_signs, sample_size=3)
        avals = list(heavy_signs.keys())

        if optimize:
            # step 2: using rough estimate where the amplitude is, use the a-values around that estimate
            a0 = np.sqrt(0.5 - 0.5 * cos_signal[0])
            avals_to_use = avals_to_usef(a0, avals, L=3)

            if disp:
                print(f'rough estimate a: {a0}')
                print(f'avals to use: {avals_to_use}')

            # step 3: now vary the signs in a sliding window of size "adjacency"
            all_signs = all_signs_to_try(avals_to_use, signs_to_try, adjacency=2)

            if disp:
                print(f'number of signs Hamming distance two: {len(all_signs)}')

            # step 4: try all the signs pick the ones that minimize the objective function
            if disp:
                print('debug')
                print(all_signs)
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)

            # step 5 (optional): do one more sweep
            hamming_distance_one_signs = list(set(generate_adjacent_sign_variations(x_star, 1)))
            x_star = minimize_obj(hamming_distance_one_signs, cos_signal, abs_sin, ula_signal, esprit, disp)
        
        else:
            # here we don't vary the signs using a sliding window, but directly use the learned sign distribution (poor performance)
            all_signs = []
            for a in avals:
                all_signs.extend(heavy_signs[a])
            all_signs = list(set(all_signs))
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)

    else:
        avals = list(heavy_signs.keys())
        signs_to_try = {}
        for a in avals:
            signs_to_try[a] = heavy_signs[a]

        if optimize:
            # step 2: using rough estimate where the amplitude is, use the a-values around that estimate
            a0 = np.sqrt(0.5 - 0.5 * cos_signal[0])
            avals_to_use = avals_to_usef(a0, avals, L=3)

            if disp:
                print(f'rough estimate a: {a0}')
                print(f'avals to use: {avals_to_use}')

            # step 3: now vary the signs in a sliding window of size "adjacency"
            all_signs = all_signs_to_try(avals_to_use, signs_to_try, adjacency=2)

            if disp:
                print(f'number of signs Hamming distance two: {len(all_signs)}')

            # step 4: try all the signs pick the ones that minimize the objective function
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp, measu)

            # step 5 (optional): do one more sweep
            hamming_distance_one_signs = list(set(generate_adjacent_sign_variations(x_star, 1)))
            x_star = minimize_obj(hamming_distance_one_signs, cos_signal, abs_sin, ula_signal, esprit, disp)
        
        else:
            # here we don't vary the signs using a sliding window, but directly use the learned sign distribution (poor performance)
            all_signs = []
            for a in avals:
                all_signs.extend(heavy_signs[a])
            all_signs = list(set(all_signs))
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)


    # Optimization is done.

    if disp:
        print(x_star)
        
    signal = cos_signal + 1.0j * x_star * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)
    theta_est = np.abs(theta_est)
    eigs = np.abs(esprit.eigs)[:2]
    basin_obj = eigs[1] - eigs[0]
    if correction:
        theta_est = apply_correction(ula_signal, measurements, theta_est, theta) #apply correction
    if disp:
        print(f'basin obj: {basin_obj}')

    cR = ula_signal.get_cov_matrix_toeplitz(csignal)
    theta_est1, _ = esprit.estimate_theta_toeplitz(cR)
    eigs = np.abs(esprit.eigs)[:2]
    true_obj = eigs[1] - eigs[0]
    if disp:
        print(f'true obj: {true_obj}')

    # compute queries required
    num_queries = np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
    max_single_query = np.max(ula_signal.depths)

    ret_dict = {'theta_est': theta_est, 'theta_est1': theta_est1, 
                'error': np.abs(np.sin(theta)-np.sin(theta_est)), 
                'error1': np.abs(np.sin(theta)-np.sin(theta_est1)), 
                'queries': num_queries, 'depth': max_single_query,
                'true_obj': true_obj, 'basin_obj': basin_obj,
                'x_star': x_star}
    
    return ret_dict

csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=True)

0.1
correct signs: [ 1.  1.  1.  1. -1.  1.  1.  1.  1.]
rough estimate a: 0.18910752115495127
avals to use: ['0.13981296510949756', '0.15634532387578837', '0.1728339924311597', '0.18927436306884374', '0.20566184157877418', '0.22199184853142423']
number of signs Hamming distance two: 114
debug
[(1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0), (1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0), (1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0), (1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0), (1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 0.0, 1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0),

{'theta_est': 0.10014050463580293,
 'theta_est1': 0.10012026433053867,
 'error': 2.6781641200374073e-05,
 'error1': 4.692056561073077e-05,
 'queries': 2600,
 'depth': 128,
 'true_obj': -397.45165984862285,
 'basin_obj': -71.49676886330401,
 'x_star': array([ 1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.])}

## Let's do some statistics now

In [18]:
a0=0.14658716786698373
signs_to_try = sample_signs(heavy_signs=heavy_signs, sample_size=3)
avals = list(heavy_signs.keys())
avals_to_usef(a0, avals, L=3), avals, a0

(['0.10663566752567158',
  '0.12324153604814797',
  '0.13981296510949756',
  '0.15634532387578837',
  '0.1728339924311597',
  '0.18927436306884374'],
 ['0.09',
  '0.10663566752567158',
  '0.12324153604814797',
  '0.13981296510949756',
  '0.15634532387578837',
  '0.1728339924311597',
  '0.18927436306884374',
  '0.20566184157877418',
  '0.22199184853142423',
  '0.23825982055751374',
  '0.2544612116232283',
  '0.2705914943005945',
  '0.286646161032655',
  '0.3026207253930916',
  '0.318510723339943',
  '0.33431171446306707',
  '0.35001928322499964',
  '0.3656290401948629',
  '0.3811366232749777',
  '0.3965376989198384',
  '0.41182796334710836',
  '0.42700314374029846',
  '0.44205899944279253',
  '0.4569913231428852',
  '0.47179594204950215',
  '0.4864687190582737',
  '0.5010055539076351',
  '0.5154023843246325',
  '0.5296551871601116',
  '0.5437599795129751',
  '0.5577128198431908',
  '0.5715098090732432',
  '0.5851470916777191',
  '0.5986208567607206',
  '0.6119273391208092',
  '0.6250628

In [19]:
np.random.seed(42)

avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [np.random.uniform(0.1, 0.9)]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [0.1, 0.2, 0.3]
avals = [np.random.uniform(0.1, 0.9) for _ in range(15)]
# narray = [2]*5 + [3]
narray = [2]*6

print('learning the distribution of signs...')
heavy_signs = get_heavy_signs(narray, int(6*len(narray)))
num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    disp=False
    # if j==4:
    #     disp=True
    for i in tqdm(range(num_mc)):
        res = csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=disp)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']

        thetas1[j][i] = res['theta_est1']
        errors1[j][i] = res['error1']

        basin_obj[j][i] = res['basin_obj']
        true_obj[j][i] = res['true_obj']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']

    print(f'constant factors query and depth: {np.percentile(errors[j], 95) * num_queries[j]}, {np.percentile(errors[j], 95) * max_single_query[j]}')

learning the distribution of signs...
theta = 0.41111546403370536


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 2.4709772661949976, 0.11801682465408944
theta = 1.0363905664439828


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 2.0856047183378665, 0.09961097162210705
theta = 0.7554209219922304


...

constant factors query and depth: 2.960967537882452, 0.14141934509289322
theta = 0.2732593578259982


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 4.931691697993268, 0.2355434840832606
theta = 0.24799415401854524


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 6.949768761548213, 0.3319292542828997


In [20]:
for i in range(len(avals)):
    print(np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i]))

0.12
0.03
0.04
0.2
0.04
0.12
0.13
0.03
0.13
0.03
0.08
0.15
0.34
0.08
0.06


In [21]:
i=6
np.array(basin_obj[i])/np.array(true_obj[i]), np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i])

(array([1.        , 0.22495293, 0.11105471, 0.03379094, 0.05337009,
        0.77715326, 0.12611136, 0.70552785, 0.76910301, 0.33074334,
        0.07567082, 1.        , 0.49445982, 1.        , 1.10845698,
        1.        , 1.        , 0.09532126, 0.60863911, 0.0182182 ,
        0.07051645, 0.48814011, 0.4470375 , 0.20416925, 0.50181441,
        0.92868392, 0.03076454, 0.46494081, 0.18230095, 1.17306871,
        0.22829146, 0.68842376, 0.03326025, 0.12765827, 0.10827588,
        0.58208381, 1.278431  , 0.35892868, 0.60215212, 0.5165016 ,
        0.09042329, 0.12780199, 0.76285183, 0.16588334, 0.60187375,
        0.192356  , 0.61966657, 0.18791708, 0.58899097, 0.65343942,
        0.08317595, 0.05017529, 0.28113619, 0.35083623, 0.1150489 ,
        1.        , 0.23433193, 1.07270974, 0.13419157, 0.04807092,
        0.11994698, 0.49255658, 0.0644887 , 0.40546557, 0.60943842,
        0.09785657, 0.14810478, 0.18114921, 0.30783823, 0.91585911,
        0.02017292, 0.96392907, 0.85925983, 1.30

In [22]:
i=6
np.sin(thetas[i]), avals[i]

(array([0.14526521, 0.14801129, 0.15046385, 0.15103016, 0.14473376,
        0.14546701, 0.1440838 , 0.14583846, 0.1499534 , 0.1447231 ,
        0.14399421, 0.1475015 , 0.14331283, 0.14046648, 0.14487862,
        0.14532385, 0.14653633, 0.14561054, 0.14491359, 0.14430642,
        0.14577509, 0.14449825, 0.1425133 , 0.14791319, 0.14863706,
        0.1435451 , 0.1439178 , 0.14811817, 0.14914262, 0.14081234,
        0.14561187, 0.14506501, 0.13834753, 0.1502974 , 0.14595509,
        0.14416454, 0.14026155, 0.14464274, 0.14432614, 0.14482318,
        0.14874697, 0.1436621 , 0.14591857, 0.14492965, 0.14256783,
        0.1444163 , 0.14138963, 0.14177751, 0.14440218, 0.14568152,
        0.14711729, 0.14454506, 0.14826789, 0.14415993, 0.14504163,
        0.14529224, 0.14737612, 0.14478538, 0.1454989 , 0.14691569,
        0.14570046, 0.14281587, 0.14835335, 0.14296055, 0.14546258,
        0.14836392, 0.13506078, 0.14245426, 0.23147936, 0.14811637,
        0.14502621, 0.14865331, 0.14519066, 0.14

In [23]:
i=6
thetas[i]/np.pi, np.arcsin(avals[i])/np.pi

(array([0.04640354, 0.0472872 , 0.04807671, 0.04825906, 0.04623257,
        0.04646847, 0.04602349, 0.04658798, 0.04791237, 0.04622914,
        0.04599467, 0.04712312, 0.0457755 , 0.04486022, 0.04627917,
        0.04642241, 0.04681253, 0.04651464, 0.04629042, 0.0460951 ,
        0.04656759, 0.04615681, 0.04551837, 0.04725562, 0.04748861,
        0.04585021, 0.04597009, 0.0473216 , 0.04765135, 0.04497142,
        0.04651507, 0.04633913, 0.04417909, 0.04802312, 0.0466255 ,
        0.04604946, 0.04479434, 0.04620329, 0.04610144, 0.04626134,
        0.04752399, 0.04588784, 0.04661375, 0.04629558, 0.0455359 ,
        0.04613044, 0.04515703, 0.04528176, 0.0461259 , 0.04653748,
        0.04699948, 0.04617186, 0.04736979, 0.04604798, 0.04633161,
        0.04641223, 0.04708277, 0.04624917, 0.04647873, 0.0469346 ,
        0.04654357, 0.04561567, 0.04739729, 0.04566221, 0.04646704,
        0.04740069, 0.04312297, 0.04549938, 0.07435657, 0.04732102,
        0.04632665, 0.04749384, 0.04637956, 0.04

In [24]:
# np.random.seed(1)

# csignal, measurements = estimate_signal(ula_signal.depths, ula_signal.n_samples, np.arcsin(avals[i]))

# plt.plot(measurements, 'r', label='measurements')
# plt.plot(np.cos((2*ula_signal.depths + 1)*theta2)**2, 'b', label='theta2')
# plt.plot(np.cos((2*ula_signal.depths + 1)*theta1)**2, 'k', label='theta1')
# plt.legend()

In [25]:
perc = 68
avg = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]}: {np.percentile(errors[i], perc) * num_queries[i]}, {np.percentile(errors[i], perc) * max_single_query[i]}')
avg

constant factors query and depth (68%) for 0.39963209507789: 1.4684292628520352, 0.07013393494218675
constant factors query and depth (68%) for 0.8605714451279329: 1.0131453447814254, 0.04838903139254569
constant factors query and depth (68%) for 0.6855951534491241: 1.085619402154012, 0.051850478908848335
constant factors query and depth (68%) for 0.5789267873576293: 1.536919682283522, 0.07340511915383986
constant factors query and depth (68%) for 0.22481491235394924: 1.3524258195636087, 0.06459347197915744
constant factors query and depth (68%) for 0.22479561626896213: 1.6642082988559777, 0.07948457546774819
constant factors query and depth (68%) for 0.14646688973455957: 1.6330656850822354, 0.07799716704870378
constant factors query and depth (68%) for 0.7929409166199481: 0.9837374556649235, 0.04698447549444411
constant factors query and depth (68%) for 0.5808920093945671: 1.4281253639962779, 0.06820897260877745
constant factors query and depth (68%) for 0.6664580622368365: 1.66036938

1.4582289590181845

In [26]:
perc = 68
avg = 0.0
for j in range(len(avals)):
    avg += 2*np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]}: {2*np.percentile(errors1[j], perc) * num_queries[j]}, {2*np.percentile(errors1[j], perc) * max_single_query[j]}')
avg

constant factors query and depth with known signs for 0.39963209507789: 4.5101565949368805, 0.2154104642357913
constant factors query and depth with known signs for 0.8605714451279329: 2.2855971542093783, 0.10916284915626881
constant factors query and depth with known signs for 0.6855951534491241: 2.41743723876647, 0.115459689015712
constant factors query and depth with known signs for 0.5789267873576293: 3.7870596498194606, 0.1808744907376459
constant factors query and depth with known signs for 0.22481491235394924: 3.9575603881457564, 0.1890178095830809
constant factors query and depth with known signs for 0.22479561626896213: 3.900315054830049, 0.18628370411128592
constant factors query and depth with known signs for 0.14646688973455957: 4.119595792406278, 0.19675681396567296
constant factors query and depth with known signs for 0.7929409166199481: 2.7747045138627473, 0.13252320066210135
constant factors query and depth with known signs for 0.5808920093945671: 3.2177930819141896, 0.

3.222632532573355

In [27]:
perc = 95
avg = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]}: {np.percentile(errors[i], perc) * num_queries[i]}, {np.percentile(errors[i], perc) * max_single_query[i]}')
avg

constant factors query and depth (95%) for 0.39963209507789: 2.4709772661949976, 0.11801682465408944
constant factors query and depth (95%) for 0.8605714451279329: 2.0856047183378665, 0.09961097162210705
constant factors query and depth (95%) for 0.6855951534491241: 2.3061130379904, 0.11014271226222806
constant factors query and depth (95%) for 0.5789267873576293: 2.5698113627945833, 0.12273725911854727
constant factors query and depth (95%) for 0.22481491235394924: 3.643809694973889, 0.17403270184949918
constant factors query and depth (95%) for 0.22479561626896213: 2.9012139605431546, 0.13856544289161335
constant factors query and depth (95%) for 0.14646688973455957: 4.027141524145057, 0.19234108772036093
constant factors query and depth (95%) for 0.7929409166199481: 1.9386990005595246, 0.09259457913120117
constant factors query and depth (95%) for 0.5808920093945671: 3.0125194985881913, 0.1438815282907793
constant factors query and depth (95%) for 0.6664580622368365: 3.8471437006328

3.6360909799463954

In [28]:
perc = 95
avg = 0.0
for j in range(len(avals)):
    avg += 2*np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]}: {2*np.percentile(errors1[j], perc) * num_queries[j]}, {2*np.percentile(errors1[j], perc) * max_single_query[j]}')
avg

constant factors query and depth with known signs for 0.39963209507789: 7.282582863218181, 0.34782485316862954
constant factors query and depth with known signs for 0.8605714451279329: 3.995912276685717, 0.19084954157304918
constant factors query and depth with known signs for 0.6855951534491241: 4.808167074791795, 0.2296438005870708
constant factors query and depth with known signs for 0.5789267873576293: 5.641087753814325, 0.2694250867493409
constant factors query and depth with known signs for 0.22481491235394924: 8.069065828703485, 0.38538821868434553
constant factors query and depth with known signs for 0.22479561626896213: 6.539721857371034, 0.31234492453115387
constant factors query and depth with known signs for 0.14646688973455957: 8.878049797408703, 0.4240262589807142
constant factors query and depth with known signs for 0.7929409166199481: 5.341488180531501, 0.25511585339851944
constant factors query and depth with known signs for 0.5808920093945671: 6.0532806783249695, 0.28

6.51479490999083

In [29]:
perc = 99
avg = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]}: {np.percentile(errors[i], perc) * num_queries[i]}, {np.percentile(errors[i], perc) * max_single_query[i]}')
avg

constant factors query and depth (99%) for 0.39963209507789: 4.151565485577929, 0.19828372468431898
constant factors query and depth (99%) for 0.8605714451279329: 2.3920036830741847, 0.11424495202742374
constant factors query and depth (99%) for 0.6855951534491241: 5.808914346302545, 0.27744068519653947
constant factors query and depth (99%) for 0.5789267873576293: 3.331572464417424, 0.1591198788975486
constant factors query and depth (99%) for 0.22481491235394924: 8.671471749402487, 0.4141598447475815
constant factors query and depth (99%) for 0.22479561626896213: 4.395075854612961, 0.20991407066808174
constant factors query and depth (99%) for 0.14646688973455957: 8.135253248694843, 0.388549408892888
constant factors query and depth (99%) for 0.7929409166199481: 2.574343569520362, 0.12295372272336057
constant factors query and depth (99%) for 0.5808920093945671: 4.0370219791711115, 0.19281299004996352
constant factors query and depth (99%) for 0.6664580622368365: 5.120727600774669, 0

6.8410830755240175

In [30]:
perc = 99
avg = 0.0
for j in range(len(avals)):
    avg += 2*np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]}: {2*np.percentile(errors1[j], perc) * num_queries[j]}, {2*np.percentile(errors1[j], perc) * max_single_query[j]}')
avg

constant factors query and depth with known signs for 0.39963209507789: 7.7149769627352045, 0.3684765116530247
constant factors query and depth with known signs for 0.8605714451279329: 5.197818002806795, 0.24825399416390662
constant factors query and depth with known signs for 0.6855951534491241: 7.7678825605658135, 0.37100334617627767
constant factors query and depth with known signs for 0.5789267873576293: 6.46265852042712, 0.30866428754278785
constant factors query and depth with known signs for 0.22481491235394924: 10.09572020016959, 0.48218365135138336
constant factors query and depth with known signs for 0.22479561626896213: 8.42257765166158, 0.4022723654524934
constant factors query and depth with known signs for 0.14646688973455957: 9.809597732443915, 0.46851810065403776
constant factors query and depth with known signs for 0.7929409166199481: 5.967617873951354, 0.285020555173796
constant factors query and depth with known signs for 0.5808920093945671: 8.661312028411515, 0.4136

8.859095198779196